<a href="https://colab.research.google.com/github/KU-HJHM/Civil-Service-Examination-DB-Project/blob/main/%EA%B3%BC%EB%AC%B8%EC%84%A0%EC%A7%91_XML_%ED%8C%8C%EC%8B%B1_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요 패키지 설치(mpire)
!pip install -U mpire

# 필요 모듈 설치(pandas, numpy, lxml 등)
import pandas as pd
import numpy as np
import lxml
# import xml.etree.ElementTree as ET
import lxml.etree as et
from tqdm import tqdm
import re
from natsort import natsorted
import glob
import multiprocessing
from mpire import WorkerPool
import gdown

# 코랩 한글 깨짐 방지
def fix_font():
    # From https://HC.Dle.pw, By Jinseo Kim
    # v1.0.0
    import os
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import matplotlib.font_manager as fm
    os.system("apt-get install -y fonts-nanum")
    os.system("fc-cache -fv")
    # mpl.font_manager._rebuild()
    # findfont = mpl.font_manager.fontManager.findfont
    # mpl.font_manager.findfont = findfont
    # mpl.backends.backend_agg.findfont = findfont
    # plt.rcParams['font.family'] = "NanumBarunGothic"
    # plt.rcParams['axes.unicode_minus'] = False
    fe = fm.FontEntry(
        fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
        name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
    fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
    plt.rcParams.update({'font.family': 'NanumGothic'}) # 폰트 설정

fix_font()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.8/272.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/KU-HJHM/Civil-Service-Examination-DB-Project
CE1000WRCHG = "/content/Civil-Service-Examination-DB-Project/과문선집 XML/과문선집 XML 03_元陵靑黃科.xml"
tree = et.parse(CE1000WRCHG)
parser=et.XMLParser(encoding='utf-8')
root = tree.getroot()

Cloning into 'Civil-Service-Examination-DB-Project'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 37 (delta 12), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 92.41 KiB | 3.08 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
metadata = root.find('Metadata')
metadata_dict = {
    'VolumeCount': metadata.find('VolumeCount').text,
    'Maintitle': metadata.find('.//Maintitle').text,
    'CompilationType': metadata.find('CompilationType').text,
    'Style': metadata.find('Style').text,
    'Institution': metadata.find('.//Institution').text,
    'Callnumber': metadata.find('.//Callnumber').text,
    'Edition': metadata.find('Edition').text,
}

formatdata = root.find('Formatdata')
formatdata_dict = {
    'Table': formatdata.find('.//Table').text,
    'PrefaceTitle': formatdata.find('.//Preface/Title').text,
    'PrefaceContent': formatdata.find('.//Preface/Content').text,
    'PrefaceAuthor': formatdata.find('.//Preface/Author').text,
}

contentdata = root.find('Contentdata')
works_data = []
for work in contentdata.findall('Work'):
    work_dict = {}
    # 시험 데이터
    exam_data = work.find('Examinationdata')
    work_dict['Exam'] = exam_data.find('Exam').text
    work_dict['Sexagenarycycle'] = exam_data.find('.//Sexagenarycycle').text
    work_dict['Year'] = exam_data.find('.//Year').text
    work_dict['Site'] = exam_data.find('.//Site').text
    work_dict['Examiner'] = ' '.join([examiner.text for examiner in exam_data.findall('.//Examiner/br')])

    # 저자 데이터
    author_data = work.find('Authordata')
    work_dict['AuthorName'] = author_data.find('.//Name').text
    work_dict['Grade'] = author_data.find('.//Grade').text

    # 답안 데이터
    answer_data = work.find('Answerdata')
    work_dict['Question'] = answer_data.find('Question').text
    work_dict['Answer'] = answer_data.find('Answer').text

    # 추가 데이터
    work_dict['Additionaldata'] = work.find('Additionaldata').text

    works_data.append(work_dict)

In [ ]:
metadata_df = pd.DataFrame([metadata_dict])
formatdata_df = pd.DataFrame([formatdata_dict])
works_df = pd.DataFrame(works_data)

In [ ]:
works_df.to_excel("works_data.xlsx", index=False)